In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
import pandas as pd
import os
import geopandas as gpd
import geoplot
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import rasterio
from statsmodels.tsa.seasonal import STL
import numpy as np
from sklearn.linear_model import LinearRegression
import datetime as dt
from matplotlib.colors import TwoSlopeNorm
import time
from rasterstats import zonal_stats
from scipy.stats import pearsonr
import plotly.express as px

#import raster_func  #import function python file

#boundary = gpd.read_file('.geojson')

/home/luckyw0w/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_5186/1581116884.py:3: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeo

In [2]:
#df = raster_func.correlation('cog/', 2015, 2021,  boundary.head(100), 'NDVI for KA districts')

In [11]:
#import boundary file 
boundary = gpd.read_file('../../../../../src/data_preprocessing/base_geojson/TS_district_boundary.json')
#reset index to get numerical values for each row
boundary = boundary.reset_index()

In [14]:
# mention directory containing all raster files
directory = 'cog/'

In [88]:
def dppd(boundary, directory, index:int):    
#take boundary geopandas, directory containg input rasters and index of geopanda dataframe as input params
    #list all files in given directory
    tiffs = os.listdir(directory)
    #sort name of files as per date in ascending order
    tiffs.sort()

    #extract dates from name of files
    dates = [pd.to_datetime(tiff[:10].replace("_", "-")) for tiff in tiffs]
    #ordinal values of dates from extracted dates from list of files
    dates_ordinal = [dt.datetime.toordinal(date) for date in dates]

    #create empty dataframe
    df = pd.DataFrame()

    #get list of mean values of raster pixels per boundary per index
    mean_list = [zonal_stats(boundary.iloc[index]["geometry"], directory+tiff, stats="mean")[0]['mean'] for tiff in tiffs]
    #create panda series of mean values and dates as an index
    ndvi = pd.Series(mean_list, index=pd.to_datetime(dates), name="NDVI")

    #Run STL
    stl = STL(ndvi,period=2, seasonal=7)
    #fit STL
    res = stl.fit()
    #list STL outputs
    res = list(res.trend)
    #store dates and res in empty dataframe
    df =pd.DataFrame()
    df['dates'] = dates_ordinal
    df['res'] = res

    #drop null values if any
    df = df.dropna()
    #Set the data in the right format for Linear Regression
    if not df.empty:
        #Set the data in the right format for Linear Regression
        x = np.array(df['dates'])
        X = x.reshape(-1, 1)
        y = np.array(df['res'])
        y = y.reshape(-1, 1)

        #Perform Linear Regression and obtain the slope
        reg = LinearRegression().fit(X, y)
        y_pred_trend = reg.predict(X)
        slope, intercept = np.polyfit(x, y_pred_trend,1)
        return slope[0]

    else:
        return 0

In [93]:
#ist dppd result for each boundary looping with index values
res = [dppd(boundary, directory, index) for index in range(len(boundary))]

In [94]:
#create np array of list
res = np.array(res)
#check min value of res, if min value is not less than 0 then there will be no negative deviance
print(np.min(res))
#normalize results: (-1,0) for negative values and (0,1) for positive values
data_norm = np.where(res >= 0, res/np.max(res), -res/np.min(res))

#create new column of deviance value
boundary['deviance'] = list(data_norm)

8.997170978427636e-06


In [95]:
#rearrange columns: geometry to last and deviance to 2nd last

col_list = list(boundary)
col_list[-1], col_list[-2] = col_list[-2], col_list[-1]
boundary = boundary[col_list]

In [96]:
boundary #output dataframe

,index,Area,Dist_Name,uid,deviance,geometry
0,0,6974.142517,Bhadradri Kothagudem,TDID1,0.459742,"POLYGON ((80.91930 17.21950, 80.93816 17.21971..."
1,1,2853.088839,Jagtial,TDID2,0.559367,"POLYGON ((79.29767 18.81833, 79.27566 18.82538..."
2,2,2112.708002,Jangoan,TDID3,0.788090,"POLYGON ((79.23083 17.99250, 79.19182 17.99887..."
3,3,4189.022904,Mulugu,TDID4,0.631274,"POLYGON ((79.84389 18.15964, 79.85469 18.14713..."
4,4,3701.834985,Kamareddy,TDID5,0.507068,"POLYGON ((78.52433 18.46778, 78.51680 18.46579..."
5,5,2127.790076,Karimnagar,TDID6,0.465008,"POLYGON ((79.12410 18.66557, 79.11729 18.66475..."
6,6,4491.247740,Khammam,TDID7,0.507913,"POLYGON ((80.91930 17.21950, 80.89326 17.22221..."
7,7,3551.035197,Mahabubabad,TDID8,0.609083,"POLYGON ((79.56249 17.61965, 79.56047 17.60609..."
8,8,3909.400107,Mancherial,TDID9,0.837950,"POLYGON ((79.93091 19.15612, 79.92258 19.16502..."
9,9,2771.683489,Medak,TDID10,0.366370,"POLYGON ((78.56878 18.18295, 78.56602 18.17606..."
